In [167]:
#%pip install -q tensorflow-recommenders
import tensorflow as tf
import pandas as pd
from tensorflow.keras.layers import Embedding, Dense
from tensorflow.keras import Sequential
import tensorflow_recommenders as tfrs
from tensorflow.keras.metrics import AUC
import math
import numpy as np



In [168]:
if tf.config.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

GPU is available


In [169]:
# Load data
basepath = '../../datasets/preprocessed_datasets/gabor/'
pandas_interaction_data = pd.read_pickle(basepath + 'user_item_interactions_only_main_products_10k.pkl')
#pandas_interaction_data = pandas_interaction_data.sample(1000)
pandas_interaction_data.amount = pandas_interaction_data.amount.apply(lambda x: 1 if x > 0 else 0) 
product_data = pd.read_pickle(basepath + 'no_product_variants_with_duplicate_ids.pkl')
pandas_interaction_data.main_product_id = pandas_interaction_data.main_product_id.astype(int)
product_data.main_product_id = product_data.main_product_id.astype(int)

display(pandas_interaction_data, product_data)

,customer_id,main_product_id,amount
325791,7347782,7023883,1
133655,4395746,4233880,1
128155,4315328,4234094,1
231063,5839435,5015580,1
42949,2070488,3480813,1
...,...,...,...
187060,5196436,4823122,1
471462,9626549,3846282,1
79554,2204900,4773983,1
158028,4807671,3849864,1


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
0,557559,4058394021466,sportliche Ballerinas Glattleder schwarz,schwarz,02.643.57,6590678,315571,w,PT,F-S|H-W,...,bis 3 cm,40.0,6.5,11.0,0.0,Gummi,0.0,0.0,"[557559, 549733, 549734, 557553, 589488, 58948...","[35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."
1,551622,4054452768212,elegante Pumps Glattleder schwarz,schwarz,05.160.37,6590677,315573,w,PT,F-S|H-W,...,3 cm - 5 cm,38.0,5.0,8.0,0.0,EVA,0.0,0.0,"[551622, 551505, 551510, 551509, 552973, 55162...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
2,547193,4054452768427,Slipper Glattleder schwarz,schwarz,04.443.27,6590677,315576,w,SK,F-S|H-W,...,3 cm - 5 cm,42.0,8.0,14.0,0.0,PU-TPU,0.0,0.0,"[547193, 547191, 547196, 547187, 547189, 54719...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
3,549776,4059701687894,Sneaker low Rauleder blau,blau,06.968.46,4997827,315567,w,VN,F-S|H-W,...,3 cm - 5 cm,39.0,6.0,10.0,0.0,Gummi-EVA,0.0,0.0,"[549776, 550599, 550596, 550593, 550601, 54978...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
4,550726,4054452851594,elegante Ballerinas Materialmix Lederimitat sc...,schwarz,06.102.67,6590678,315574,w,SK,F-S|H-W,...,bis 3 cm,37.5,4.5,7.0,0.0,EVA,0.0,0.0,"[550726, 550736, 550735, 550732, 550728, 55072...","[35.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3292,10603553,4066558951733,Sandale met plateauzool Suède blauw,blauw,24.764.36,6590677,315568,w,SK,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,PU-TPU,0.0,0.0,"[10603553, 10544662]","[35.0, 37.0]"
3293,10442948,4065171827272,Mokassin Materialmix Leder pink,pink,26.090.21,6590678,315590,w,PT,F-S,...,bis 3 cm,40.5,7.0,12.0,0.0,Gummi,0.0,0.0,"[10442948, 10442949]","[40.5, 41.0]"
3294,10782704,4251234499207,Shopper ANDIE blau,blau,921453,363013,363017,w,DE,F-S,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,[10782704],[]
3295,10679703,4066558303617,Elegante pumps Glad leer wit,wit,21.450.60,6590677,315573,w,PT,F-S,...,5 cm - 8 cm,35.0,2.5,3.0,0.0,TPU,0.0,0.0,[10679703],[35.0]


In [170]:
# Create a tf.data.Dataset from the interaction data
interaction_dataset = tf.data.Dataset.from_tensor_slices({
    'customer_id': pandas_interaction_data['customer_id'].values,
    'main_product_id': pandas_interaction_data['main_product_id'].values,
    'amount': pandas_interaction_data['amount'].values
})
#variables 
dataset_len = pandas_interaction_data.shape[0]
test_len = math.ceil(dataset_len * 0.2)
train_len = dataset_len - test_len

metrics_batchsize = 1024
dimension = 128
train_batch_size = dimension
test_batch_size = int(dimension/2)
random_seed = 27

In [171]:
#train test split
tf.random.set_seed(random_seed)
# shuffled = interaction_dataset.shuffle(len, seed=random_seed, reshuffle_each_iteration=False)
# train = shuffled.take(train_len)
# test = shuffled.skip(train_len).take(test_len)
shuffled = interaction_dataset.shuffle(dataset_len, seed=random_seed, reshuffle_each_iteration=False)

train = shuffled.take(train_len)
test = shuffled.skip(train_len).take(test_len)
display(train, test)

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

<_TakeDataset element_spec={'customer_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'main_product_id': TensorSpec(shape=(), dtype=tf.int64, name=None), 'amount': TensorSpec(shape=(), dtype=tf.int64, name=None)}>

In [172]:
# Unique customer and product identifiers
unique_customer_ids = np.array(pandas_interaction_data["customer_id"].unique())
unique_product_ids = np.array(pandas_interaction_data["main_product_id"].unique())

user_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
user_ids_vocabulary.adapt(unique_customer_ids)

product_ids_vocabulary = tf.keras.layers.IntegerLookup(mask_token=None)
product_ids_vocabulary.adapt(unique_product_ids)
# print(unique_product_ids.shape[0], unique_customer_ids.shape[0], unique_product_ids, unique_product_ids)

In [173]:
# Define a model using TensorFlow Recommenders
product_ids_dataset = tf.data.Dataset.from_tensor_slices(unique_product_ids)
# Define a model using TensorFlow Recommenders
product_ids_dataset = tf.data.Dataset.from_tensor_slices(unique_product_ids)
class RetailModel(tfrs.Model):
    def __init__(self, unique_product_ids_vocab, unique_customer_ids_vocab, embedding_dimension=32, metrics_batchsize=1024,
                 retrieval_loss_weight=1.0, ranking_loss_weight=1.0):
        super().__init__()
        self.retrieval_loss_weight = retrieval_loss_weight
        self.ranking_loss_weight = ranking_loss_weight
        # Embedding layers
        self.user_embedding = tf.keras.Sequential([
            unique_customer_ids_vocab,
            tf.keras.layers.Embedding(input_dim=unique_customer_ids_vocab.vocabulary_size(), output_dim=embedding_dimension)
        ])
        self.product_embedding = tf.keras.Sequential([
            unique_product_ids_vocab,
            tf.keras.layers.Embedding(input_dim=unique_product_ids_vocab.vocabulary_size(), output_dim=embedding_dimension)
        ])
        # Dense layer for transforming embeddings
        self.dense_layer = tf.keras.layers.Dense(128, activation="relu")

        # Setup retrieval task and metrics
        self.retrieval_task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(candidates=product_ids_dataset.batch(metrics_batchsize).map(self.product_model))
        )

        # Setup ranking task with pairwise hinge loss
        self.ranking_task = tfrs.tasks.Ranking(
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=[tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.AUC()]
        )

    def product_model(self, product_ids):
        return self.product_embedding(product_ids)

    def dot_product_score(self, user, product):
        """Computes the dot product between user and product embeddings to get the interaction score."""
        return tf.reduce_sum(user * product, axis=1)

    def compute_loss(self, features, training=False):
        # Extract embeddings
        user_embeddings = self.user_embedding(features["customer_id"])
        product_embeddings = self.product_embedding(features["main_product_id"])

        # Apply dense layers
        user_output = self.dense_layer(user_embeddings)
        product_output = self.dense_layer(product_embeddings)

        # Compute scores
        positive_logits = self.dot_product_score(user_output, product_output)
        # Compute losses for both retrieval and ranking
        retrieval_loss = self.retrieval_task(user_output, product_output, compute_metrics=not training)
        ranking_loss = self.ranking_task(labels=features["amount"], predictions=positive_logits, sample_weight=features.get("weight"))

        return (self.retrieval_loss_weight * retrieval_loss +
                self.ranking_loss_weight * ranking_loss)

    def evaluate(self, validation_dataset, *args, **kwargs):
        return super().evaluate(validation_dataset, *args, **kwargs)
        # or append additional results as needed



In [174]:
model = RetailModel(product_ids_vocabulary, user_ids_vocabulary, embedding_dimension=128, retrieval_loss_weight=0.5, ranking_loss_weight=0.8, metrics_batchsize=metrics_batchsize)

initial_learning_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.96,
    staircase=True
)

model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), loss='categorical_crossentropy') # using legacy instead of tf.keras.optimizers.Adagrad, because newer version is slow on m1/m2 macs

In [175]:
# Shuffle, batch, and cache the data.
cached_train = train.shuffle(dataset_len).batch(train_batch_size).cache()
cached_test = test.batch(test_batch_size).cache()
# Train the model
# model.fit(cached_train, validation_data=cached_test, validation_freq=5, epochs=3)
model.fit(cached_train, validation_freq=5, epochs=10)

Epoch 1/10
63/63 [==============================] - 15s 167ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - root_mean_squared_error: 0.9981 - categorical_accuracy: 0.0159 - auc_8: 0.0000e+00 - loss: 1198.1753 - regularization_loss: 0.0000e+00 - total_loss: 1198.1753
Epoch 2/10
63/63 [==============================] - 4s 63ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_5_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_10_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_50_categorical_accuracy: 0.0000e+00 - factorized_top_k/top_100_categorical_accuracy: 0.0000e+00 - root_mean_squared_error: 1.0128 - categorical_accuracy: 0.0000e+00 - auc_8: 0.0000e+00 - loss: 973.6829 - regularizat

In [176]:
# Evaluate the model
k = 5
result_evaluate_train = model.evaluate(cached_train)
result_evaluate_train

63/63 [==============================] - 41s 616ms/step - factorized_top_k/top_1_categorical_accuracy: 0.7107 - factorized_top_k/top_5_categorical_accuracy: 0.8827 - factorized_top_k/top_10_categorical_accuracy: 0.9095 - factorized_top_k/top_50_categorical_accuracy: 0.9294 - factorized_top_k/top_100_categorical_accuracy: 0.9317 - root_mean_squared_error: 0.9226 - categorical_accuracy: 0.0000e+00 - auc_8: 0.0000e+00 - loss: 828.8191 - regularization_loss: 0.0000e+00 - total_loss: 828.8191


[0.7107499837875366,
 0.8827499747276306,
 0.909500002861023,
 0.9293749928474426,
 0.9317499995231628,
 0.922574520111084,
 0.0,
 0.0,
 345.8175048828125,
 0,
 345.8175048828125]

In [177]:
result_evaluat_test = model.evaluate(cached_test)
result_evaluat_test #rsme 3.3579

32/32 [==============================] - 21s 664ms/step - factorized_top_k/top_1_categorical_accuracy: 0.7815 - factorized_top_k/top_5_categorical_accuracy: 0.9635 - factorized_top_k/top_10_categorical_accuracy: 0.9780 - factorized_top_k/top_50_categorical_accuracy: 0.9905 - factorized_top_k/top_100_categorical_accuracy: 0.9940 - root_mean_squared_error: 0.9164 - categorical_accuracy: 0.0312 - auc_8: 0.0000e+00 - loss: 331.7306 - regularization_loss: 0.0000e+00 - total_loss: 331.7306


[0.781499981880188,
 0.9635000228881836,
 0.9779999852180481,
 0.9904999732971191,
 0.9940000176429749,
 0.9164440035820007,
 0.03125,
 0.0,
 58.537841796875,
 0,
 58.537841796875]

In [178]:
print("Name\t\tValue")
print("-" * 30)
for metric in model.metrics:
    print(f"{metric.name}\t\t{metric.result().numpy()}")
#print(f"AUC: {model.auc_metric.result().numpy()}")

Name		Value
------------------------------
factorized_top_k/top_1_categorical_accuracy		0.781499981880188
factorized_top_k/top_5_categorical_accuracy		0.9635000228881836
factorized_top_k/top_10_categorical_accuracy		0.9779999852180481
factorized_top_k/top_50_categorical_accuracy		0.9904999732971191
factorized_top_k/top_100_categorical_accuracy		0.9940000176429749
root_mean_squared_error		0.9164440035820007
categorical_accuracy		0.03125
auc_8		0.0


In [179]:
sample = next(iter(cached_test.take(1)))

# Extract customer_id from the sample
customer_id = sample['customer_id'].numpy()
customer_id

array([ 4275416,  5191936,  9181415,  2035007,  8891134, 10874927,
        8610367,  3736938,  1957983,  6480012,  1958913,  8908474,
        7285710,  8867556,  6373394,  2196344,  8446599,  1948734,
        6525510,  7553474,  9446582,  9965105,  2142504, 10860996,
        8121725, 10145415,  9108029, 10511849, 10170223, 10270040,
        6456133,  5530970,  3959734,  7019456,  9736062,  4100439,
        5318358,  9834638,  6798151,  2116409,  2032193,  8118166,
        7410488,  8147869,  9441353,  9971211,  8582644,  8657464,
        6799071, 10017324,  8062560,  4460873,  1992810,  7887224,
        9153335,  2006015,  8272020, 10112803,  6568213,  4336120,
        2083624, 10633283,  2201167,  8185718])

In [180]:
def get_random_customer_from_test_data():
    return next(iter(test.take(1)))['customer_id'].numpy()

def display_item_ids(item_ids):
    display(product_data[product_data.main_product_id.isin(item_ids) ])

def display_products_by_customer_id(customer_id):
    display_item_ids(pandas_interaction_data[pandas_interaction_data.customer_id ==  customer_id].main_product_id.tolist())
    

index = tfrs.layers.factorized_top_k.BruteForce(model.user_embedding, k=5)
index.index_from_dataset(
    product_ids_dataset.batch(100).map(lambda id: (id, model.product_model(id))))

def predict_user(user_id):
    print('predicting user: ', user_id)
    print('user already bought following products: ')
    display_products_by_customer_id(user_id)
    
    score, predicted_product_ids = index(np.array([user_id]))
    
    print('predicted products: ')
    display_item_ids(predicted_product_ids[0].numpy())
    print('scores: ', score[0].numpy())


user_id = get_random_customer_from_test_data()

predict_user(user_id)

predicting user:  4275416
user already bought following products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
512,4134223,4062862442301,Elegante Stiefelette Rauleder gelb,gelb,51.800.13,6590677,5014851,w,SK,H-W,...,3 cm - 5 cm,38.0,5.0,8.0,23.0,EVA,0.0,0.0,"[4134223, 4134220, 4222459, 4134229, 4222456, ...","[35.0, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."


predicted products: 


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
905,5150562,4062862552161,Trekkingsandale Rauleder grün,grün,66.826.42,4997827,315596,w,VN,F-S,...,3 cm - 5 cm,40.0,6.5,11.0,0.0,Gummi-EVA,0.0,0.0,"[5150562, 5150563, 5150564, 5150559, 5150566, ...","[35.0, 36.0, 37.0, 37.5, 38.0, 38.5, 39.0, 40...."
1200,6112985,4064032609262,Plateau Pumps Lederimitat beige,beige,71.270.42,6590677,315569,w,PT,H-W,...,8 cm - 10 cm,37.0,4.0,6.0,0.0,EVA,1.0,1.0,"[6112985, 6112993, 6112989, 6112995, 6112988, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
1313,6111404,4064032804575,Biker- / Combat Boot Rauleder,,71.711.11,6590677,5014852,w,PT,H-W,...,5 cm - 8 cm,43.0,9.0,16.0,16.0,TR,1.0,1.0,"[6111404, 6684597, 6555833, 6570295, 6555835, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
1855,7811299,4064032132418,Pantolette Materialmix Leder blau,blau,83.740.90,6590677,315572,w,PT,F-S,...,3 cm - 5 cm,42.0,8.0,10.0,0.0,TR,0.0,0.0,"[7811299, 7791279, 7791384, 7791284, 7791382, ...","[36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43...."
1951,7800773,4064032948538,Slip-on Suède beige,beige,84.213.14,6590677,315576,w,PT,F-S,...,bis 3 cm,43.0,9.0,16.0,0.0,TPU,1.0,1.0,"[7800773, 7799248, 7800435, 8531254, 8531265, ...","[37.0, 38.0, 39.0, 40.0, 40.5, 41.0, 43.0, 44.0]"


scores:  [0.04437084 0.03361784 0.02974563 0.0293104  0.02853333]


In [181]:
def calculate_coverage_and_scores(user_ids, total_items_count, example_user_id):
    all_scores = []
    all_predicted_product_ids = []  # Use a set to track unique recommended items

    for user_id in user_ids:
        # Call the prediction function for each user
        score, predicted_product_ids = index(np.array([user_id]))
        if user_id == example_user_id:
            display('predicting user: ', user_id)
            display('predicted products: ')
            display(score[0].numpy(), predicted_product_ids[0].numpy())
            display_products_by_customer_id(user_id)
            display_item_ids(predicted_product_ids[0].numpy())
        # Append the scores to the all_scores list
        all_scores.extend(score[0].numpy())

        # Update the set of all unique predicted product IDs
        all_predicted_product_ids.extend(predicted_product_ids[0].numpy())

    # Calculate min, max, and average scores
    min_score = np.min(all_scores)
    max_score = np.max(all_scores)
    avg_score = np.mean(all_scores)

    # Calculate item coverage and catalog coverage
    num_unique_items = len(np.unique(all_predicted_product_ids))
    catalog_coverage = num_unique_items / total_items_count

    # Print the results
    print(f"Minimum Score: {min_score}")
    print(f"Maximum Score: {max_score}")
    print(f"Average Score: {avg_score}")
    print(f"Catalog Coverage: {catalog_coverage:.2%} (Unique recommended items {num_unique_items} / Total catalog items {total_items_count})")

    return min_score, max_score, avg_score, catalog_coverage, all_predicted_product_ids, num_unique_items, total_items_count

# Example usage
total_items_count = len(pandas_interaction_data.main_product_id.unique())

user_ids = pandas_interaction_data.customer_id.unique()
test_user_id = 7347782
min_score, max_score, avg_score, catalog_coverage, all_predicted_product_ids, num_unique_items, total_items_count = calculate_coverage_and_scores(user_ids, total_items_count, test_user_id)

'predicting user: '

7347782

'predicted products: '

array([0.04083208, 0.03995267, 0.03527907, 0.03274186, 0.03223534],
      dtype=float32)

array([ 3695057,  6177761,  4125488,  7589688, 10325023])

,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
1581,7023883,4064032477885,Winterstiefelette Glattleder,,72.802.93,6590678,315580,w,PT,H-W,...,5 cm - 8 cm,44.0,9.5,17.0,26.0,TR,0.0,0.0,"[7023883, 7023906, 7016032, 6902928, 7023753, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."


,main_product_id,productNumber,productName,productColorName,articleNumber,brand__id,mainCategory__id,gender,originCountry,line,...,heelHeightGroup,sizeEu,sizeUk,sizeIndex,shaftLength__value,sole,isTransferee,isSuccessor,duplicate_product__ids,product_sizes
419,3695057,4062862077640,PG1008 Sneaker low Glattleder weiß,weiß,1008.10.01,3692257,3692261,m,PT,H-W,...,bis 3 cm,46.5,11.5,14.0,0.0,TR,0.0,0.0,"[3695057, 3695047, 3695055, 3695060, 3695051, ...","[40.0, 40.5, 42.5, 44.5, 45.0, 46.5, 48.5, 49.5]"
491,4125488,4062862356011,Elegante Stiefelette Rauleder beige,beige,56.611.32,6590678,5014851,w,PT,H-W,...,3 cm - 5 cm,38.5,5.5,9.0,14.0,Gummi,0.0,0.0,"[4125488, 4125489, 4125494, 4125496, 4125495, ...","[35.5, 37.0, 37.5, 38.0, 38.5, 39.0, 40.0, 40...."
1203,6177761,4064032767115,PG1022 Sneaker low Glattleder schwarz,schwarz,1022.11.08,3692257,3692261,m,PT,H-W,...,3 cm - 5 cm,45.0,10.5,12.0,0.0,TR,0.0,0.0,"[6177761, 6177754, 6177758, 6177759, 6177762, ...","[40.0, 40.5, 41.0, 42.0, 42.5, 43.0, 44.0, 44...."
1702,7589688,4064032924808,Eleganter Ballerina Rauleder beige,beige,81.302.12,6590677,315576,w,PT,F-S,...,bis 3 cm,38.5,5.5,9.0,0.0,TPU,1.0,1.0,"[7589688, 7589689, 7589687, 7590119, 7589686, ...","[35.0, 35.5, 36.0, 37.0, 37.5, 38.0, 38.5, 39...."
2970,10325023,4065171996220,Pantolette Rauleder pink,pink,22.880.44,6590678,315572,w,PT,F-S,...,3 cm - 5 cm,35.0,2.5,3.0,0.0,TR,0.0,0.0,"[10325023, 10224836, 10224838, 10224835, 10224...","[35.0, 36.0, 37.0, 38.0, 38.5, 39.0, 40.0, 40...."


Minimum Score: 0.021564090624451637
Maximum Score: 0.21329368650913239
Average Score: 0.03643861785531044
Catalog Coverage: 100.00% (Unique recommended items 1377 / Total catalog items 1377)


In [182]:
display(f"min score: {min_score}, max score: {max_score}, avg_score: {avg_score}, catalog_coverage: {catalog_coverage}, min score: {min_score},")

'min score: 0.021564090624451637, max score: 0.21329368650913239, avg_score: 0.03643861785531044, catalog_coverage: 1.0, min score: 0.021564090624451637,'

In [183]:
# 3 epochs: min score: 0.028976473957300186, max score: 0.31238240003585815, avg_score: 0.09717307984828949, catalog_coverage: 0.8976034858387799, min score: 0.028976473957300186,
# 10 epochs: min score: 0.03581412881612778, max score: 0.41449666023254395, avg_score: 0.16217400133609772, catalog_coverage: 0.9201161946259986, min score: 0.03581412881612778,
# 100 epochs: min score: 0.04029332101345062, max score: 0.6680517196655273, avg_score: 0.21841196715831757, catalog_coverage: 0.9201161946259986, min score: 0.04029332101345062,

In [184]:
#print(all_predicted_product_ids)

In [185]:
display("predicted items", len(list(set(all_predicted_product_ids))), "unique items", num_unique_items, "number of items", total_items_count)

'predicted items'

1377

'unique items'

1377

'number of items'

1377